In [1]:
import pandas as pd
import numpy as np
from dateutil import parser
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Multiply, Embedding, Dot, Flatten, Lambda, Concatenate, Add
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow import random_normal_initializer
from tensorflow.keras.constraints import NonNeg
from sklearn.cluster import KMeans

In [2]:
df_orders = pd.read_csv("olist_orders_dataset.csv")
df_order_items = pd.read_csv("olist_order_items_dataset.csv")
df_customers = pd.read_csv("olist_customers_dataset.csv")
df_products = pd.read_csv("olist_products_dataset.csv")
df_reviews = pd.read_csv("olist_order_reviews_dataset.csv")

In [3]:
df_orders.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [4]:
df_order_items.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [5]:
df_joined = pd.merge(df_orders, df_order_items, on="order_id")
df_joined = pd.merge(df_joined, df_reviews, on="order_id")
df_joined = pd.merge(df_joined, df_customers, on="customer_id")
df_joined = pd.merge(df_joined, df_products, on="product_id")

In [6]:
df_joined.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,...,customer_city,customer_state,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,1,87285b34884572647811a353c7ac498a,...,sao paulo,SP,utilidades_domesticas,40.0,268.0,4.0,500.0,19.0,8.0,13.0
1,128e10d95713541c87cd1a2e48201934,a20e8105f23924cd00833fd87daa0831,delivered,2017-08-15 18:29:31,2017-08-15 20:05:16,2017-08-17 15:28:33,2017-08-18 14:44:43,2017-08-28 00:00:00,1,87285b34884572647811a353c7ac498a,...,sao paulo,SP,utilidades_domesticas,40.0,268.0,4.0,500.0,19.0,8.0,13.0
2,0e7e841ddf8f8f2de2bad69267ecfbcf,26c7ac168e1433912a51b924fbd34d34,delivered,2017-08-02 18:24:47,2017-08-02 18:43:15,2017-08-04 17:35:43,2017-08-07 18:30:01,2017-08-15 00:00:00,1,87285b34884572647811a353c7ac498a,...,sao paulo,SP,utilidades_domesticas,40.0,268.0,4.0,500.0,19.0,8.0,13.0
3,bfc39df4f36c3693ff3b63fcbea9e90a,53904ddbea91e1e92b2b3f1d09a7af86,delivered,2017-10-23 23:26:46,2017-10-25 02:14:11,2017-10-27 16:48:46,2017-11-07 18:04:59,2017-11-13 00:00:00,1,87285b34884572647811a353c7ac498a,...,florianopolis,SC,utilidades_domesticas,40.0,268.0,4.0,500.0,19.0,8.0,13.0
4,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,1,595fac2a385ac33a80bd5114aec74eb8,...,barreiras,BA,perfumaria,29.0,178.0,1.0,400.0,19.0,13.0,19.0


In [7]:
df_final = df_joined[['customer_id', 'product_id', 'review_score', 'order_purchase_timestamp', 'customer_city', 'product_category_name']]
df_final.head()

,customer_id,product_id,review_score,order_purchase_timestamp,customer_city,product_category_name
0,9ef432eb6251297304e76186b10a928d,87285b34884572647811a353c7ac498a,4,2017-10-02 10:56:33,sao paulo,utilidades_domesticas
1,a20e8105f23924cd00833fd87daa0831,87285b34884572647811a353c7ac498a,4,2017-08-15 18:29:31,sao paulo,utilidades_domesticas
2,26c7ac168e1433912a51b924fbd34d34,87285b34884572647811a353c7ac498a,5,2017-08-02 18:24:47,sao paulo,utilidades_domesticas
3,53904ddbea91e1e92b2b3f1d09a7af86,87285b34884572647811a353c7ac498a,3,2017-10-23 23:26:46,florianopolis,utilidades_domesticas
4,b0830fb4747a6c6d20dea0b8c802d7ef,595fac2a385ac33a80bd5114aec74eb8,4,2018-07-24 20:41:37,barreiras,perfumaria


In [8]:
city_count = df_final['customer_city'].nunique()
category_count = df_final['product_category_name'].nunique()
user_count = df_final['customer_id'].nunique()
item_count = df_final['product_id'].nunique()
time_bins_count = 26
tracked_categories_count = 3


In [9]:
customer_name_to_id = {customer_name:index for index, customer_name in enumerate(df_final['customer_id'].unique())}
customer_id_to_name = {index:customer_name for index, customer_name in enumerate(df_final['customer_id'].unique())}

product_name_to_id = {product_name:index for index, product_name in enumerate(df_final['product_id'].unique())}
product_id_to_name = {index:product_name for index, product_name in enumerate(df_final['product_id'].unique())}

location_name_to_id = {location_name:index for index, location_name in enumerate(df_final['customer_city'].unique())}
location_id_to_name = {index:location_name for index, location_name in enumerate(df_final['customer_city'].unique())}

category_name_to_id = {category_name:index for index, category_name in enumerate(df_final['product_category_name'].unique())}
category_id_to_name = {index:category_name for index, category_name in enumerate(df_final['product_category_name'].unique())}

In [10]:
df_train = df_final.copy()
df_train['customer_id'] = df_train['customer_id'].apply(lambda x: customer_name_to_id[x])
df_train['product_id'] = df_train['product_id'].apply(lambda x: product_name_to_id[x])
df_train['customer_city'] = df_train['customer_city'].apply(lambda x: location_name_to_id[x])
df_train['product_category'] = df_train['product_category_name'].apply(lambda x: category_name_to_id[x])
df_train = df_train.drop(columns=['product_category_name'])
df_train.head()

,customer_id,product_id,review_score,order_purchase_timestamp,customer_city,product_category
0,0,0,4,2017-10-02 10:56:33,0,0
1,1,0,4,2017-08-15 18:29:31,0,0
2,2,0,5,2017-08-02 18:24:47,0,0
3,3,0,3,2017-10-23 23:26:46,1,0
4,4,1,4,2018-07-24 20:41:37,2,1


In [11]:
start_date = df_orders['order_purchase_timestamp'].min()
end_date = df_orders['order_purchase_timestamp'].max()

df_train['time_bin'] = df_train['order_purchase_timestamp'].apply(lambda x: int((parser.parse(end_date)-parser.parse(x)).days / 30))
df_train = df_train.drop(columns=['order_purchase_timestamp'])
df_train.describe()['time_bin']

count    113322.000000
mean          9.173011
std           5.105415
min           1.000000
25%           5.000000
50%           9.000000
75%          13.000000
max          25.000000
Name: time_bin, dtype: float64

In [12]:
df_train.head()

,customer_id,product_id,review_score,customer_city,product_category,time_bin
0,0,0,4,0,0,12
1,1,0,4,0,0,14
2,2,0,5,0,0,14
3,3,0,3,1,0,11
4,4,1,4,2,1,2


In [13]:
# Calculate favourite product categories for every user
df_favourite_categories =  df_train.groupby(['customer_id']).apply(lambda x: x.groupby(['product_category']).size().reset_index(name='size').nlargest(tracked_categories_count,'size').drop(columns=['size']))

user_favourite_categories = np.zeros((user_count, tracked_categories_count))
for user_index in range(user_count):
    try:
        favourite_categories = df_favourite_categories.loc[user_index].values.reshape(-1).tolist()
        if len(favourite_categories) < tracked_categories_count:
            favourite_categories.extend([favourite_categories[0]] * (tracked_categories_count - len(favourite_categories)))
        user_favourite_categories[user_index] = favourite_categories
    except KeyError:
        pass

In [80]:
class MatrixFactorization:
    def __init__(self,  item_count, category_count, location_count, time_bins_count, embedding_category_dim = 30, embedding_location_dim=10,
                 favourite_categories_count=3, regularization_scale = 1e-3, epochs = 50, batchsize = 256,
                 learning_rate=0.001, beta_1=0.9, beta_2=0.999, validation_split=0.2):
        self.item_count = item_count
        self.time_bins_count = time_bins_count
        self.category_count = category_count
        self.location_count = location_count
        self.embedding_category_dim = embedding_category_dim
        self.embedding_location_dim = embedding_location_dim
        self.embedding_dim = embedding_category_dim + embedding_location_dim
        self.favourite_categories_count = favourite_categories_count
        self.regularization_scale = regularization_scale
        self.epochs = epochs
        self.batchsize = batchsize
        self.validation_split = validation_split
        self.optimizer = Adam(learning_rate=learning_rate, beta_1=beta_1, beta_2=beta_2)

        self.build_model()

    # Build prediction model
    def build_model(self):
        self.userInputLocation = Input(shape = (1, ), dtype = "int32", name="input_location") # Define user location index input (integer)
        self.userInputCategories = Input(shape = (self.favourite_categories_count, ), dtype = "int32", name="input_categories") # Define favourite category indexes input (array of integers)
        self.itemInputTimeBin = Input(shape = (1, ), dtype = "int32", name="input_time_bin") # Define time bin index input (integer)
        self.itemInputIndex = Input(shape = (1, ), dtype = "int32", name="input_item_index") # Define item index input (integer)
        
        l2_loss = l2(self.regularization_scale)
        initializer = random_normal_initializer()
        constraints = NonNeg()


        self.categoryEmbeddingLayer = Embedding(input_dim = self.category_count+1, output_dim = self.embedding_category_dim,
                                                input_length = self.favourite_categories_count,
                                                embeddings_regularizer=l2_loss,
                                                embeddings_initializer=initializer,
                                                embeddings_constraint=constraints,
                                                name="category_embedding")

        self.userCategoriesEmbedding = Lambda(lambda x: K.mean(x, axis=1), name="average_category_embedding")(
            self.categoryEmbeddingLayer(self.userInputCategories)
        )

        self.locationEmbeddingLayer = Embedding(input_dim = self.location_count+1, output_dim = self.embedding_location_dim,
                                                input_length = 1,
                                                embeddings_regularizer=l2_loss,
                                                embeddings_initializer=initializer,
                                                embeddings_constraint=constraints,
                                                name="city_embedding")

        self.userLocationEmbedding = Flatten(name="flattened_city_embedding")(
            self.locationEmbeddingLayer(self.userInputLocation)
        )

        self.userEmbedding = Concatenate(axis=1, name="user_embedding")(
            [self.userCategoriesEmbedding, self.userLocationEmbedding]
        )
        

        self.itemEmbeddingLayer = Embedding(input_dim = self.item_count+1, output_dim = self.embedding_dim,
                                            input_length = 1,
                                            embeddings_regularizer = l2_loss,
                                            embeddings_initializer = initializer,
                                            embeddings_constraint=constraints,
                                            name="item_embedding")

        self.itemEmbeddingBias = Embedding(input_dim = self.item_count+1, output_dim = 1,
                                           input_length = 1,
                                           embeddings_regularizer = l2_loss,
                                           embeddings_initializer = initializer,
                                           name="item_embedding_bias")
        self.itemBias = self.itemEmbeddingBias(self.itemInputIndex)

        self.timeEmbeddingIndex = Lambda(lambda x: x[0]*self.time_bins_count + x[1]) (
            [self.itemInputIndex, self.itemInputTimeBin]
        )

        self.itemEmbeddingTime = Embedding(input_dim = self.item_count*time_bins_count+1, output_dim = 1,
                                           input_length = 1,
                                           embeddings_regularizer = l2_loss,
                                           embeddings_initializer = initializer,
                                           name="item_embedding_bias_time")

        self.itemTimeBias = self.itemEmbeddingTime(self.itemInputTimeBin)

        self.itemEmbedding = self.itemEmbeddingLayer(self.itemInputIndex)

        self.dotLayer = Dot(axes = -1, name="dot_product_layer")(
            [self.userEmbedding,
             self.itemEmbedding]
        )

        self.output = Add(name="final_output")([
                self.dotLayer,
                self.itemBias,
                self.itemTimeBias
        ])


        self.model = Model(inputs = [self.userInputCategories, self.userInputLocation, self.itemInputTimeBin, self.itemInputIndex], outputs = self.output)
        self.model.compile(optimizer = self.optimizer, loss = "mean_squared_error")

    def fit(self, df, fav_categories):
        categories = fav_categories[df['customer_id']]
        cities = df['customer_city'] 
        time_bins = df['time_bin']
        items = df['product_id']
        scores = df['review_score']        

        history = self.model.fit([categories, cities, time_bins, items], scores, epochs = self.epochs, batch_size = self.batchsize,
                    validation_split = self.validation_split, shuffle=True)


        self.userEmbeddingOutputModel = Model(inputs = [self.userInputCategories, self.userInputLocation], outputs = self.userEmbedding)
        self.itemEmbeddingOutputModel = Model(inputs = [self.itemInputTimeBin, self.itemInputIndex], outputs = [self.itemEmbedding, self.itemBias, self.itemTimeBias])


        userEmbeddingInput = Input(shape = (self.embedding_dim, ))
        customDotLayer = Dot(axes = -1, name="dot_product_layer")([userEmbeddingInput,
                                   self.itemEmbedding])

        self.output = Add(name="final_output")([
                customDotLayer,
                self.itemBias,
                self.itemTimeBias
        ])
        self.itemRecommendationOutputModel = Model(inputs = [userEmbeddingInput, self.itemInputTimeBin, self.itemInputIndex], outputs = [self.itemEmbedding, self.itemBias, self.itemTimeBias, self.output])

        return history

    def recommend(self, fav_categories, user_city, current_date, initial_recommendation_size=100, recommendation_size=10):
        time_bin = int((parser.parse(end_date)-parser.parse(current_date)).days / 30)
        input_time_bin = np.full((item_count, 1), time_bin)
        item_input_vector = np.zeros((item_count, 1))
        for item_index in range(item_count):
            item_input_vector[item_index, 0] = item_index


        if fav_categories is None: # Make a recommendation to a user who is not logged in
            items_embeddings, items_biases, items_time_biases = self.itemEmbeddingOutputModel([input_time_bin, item_input_vector])
            scores = items_biases+items_time_biases
        else:

            fav_categories = list(map(lambda x: category_name_to_id[x], fav_categories))
            fav_categories = np.array(fav_categories).reshape(1, -1)
            user_city = location_name_to_id[user_city]
            user_city = np.array(user_city).reshape(1, -1)
            user_embedding = self.userEmbeddingOutputModel([fav_categories, user_city])

            user_embedding_vector = np.repeat(user_embedding, item_count, axis=0)
            items_embeddings, items_biases, items_time_biases, scores = self.itemRecommendationOutputModel([user_embedding_vector, input_time_bin, item_input_vector])


        items_embeddings = items_embeddings.numpy()
        scores = scores.numpy()
        best_indexes = ((-scores.flatten()).argsort())[:initial_recommendation_size]
        diversified_indexes = self.diversify(items_embeddings, scores, best_indexes, recommendation_size)

        final_product_ids = [product_id_to_name[product_id] for product_id in diversified_indexes]

        return final_product_ids, scores

    def diversify(self, items_embeddings, scores, best_indexes, recommendation_size):
        scores = scores[best_indexes]
        items_embeddings = items_embeddings.reshape(-1, self.embedding_dim)
        labels = KMeans(recommendation_size).fit_predict(items_embeddings[best_indexes])
        predictions = []
        for cluster_index in range(recommendation_size):
            predictions.append(np.argwhere(labels == cluster_index)[0][0])
        return np.take(best_indexes, predictions)

In [81]:
mf = MatrixFactorization(item_count, category_count, city_count, time_bins_count, embedding_category_dim=40, embedding_location_dim=20, batchsize=512, epochs=70)

In [82]:
mf.fit(df_train, user_favourite_categories)

Epoch 1/70
d:\python36\lib\site-packages\tensorflow\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
178/178 [==============================] - 5s 27ms/step - loss: 16.8676 - val_loss: 17.1696
Epoch 2/70
178/178 [==============================] - 6s 36ms/step - loss: 11.3047 - val_loss: 17.1490
Epoch 3/70
178/178 [==============================] - 5s 30ms/step - loss: 8.7816 - val_loss: 16.8677
Epoch 4/70
178/178 [==============================] - 5s 27ms/step - loss: 7.5338 - val_loss: 16.3749
Epoch 5/70
178/178 [==============================] - 5s 28ms/step - loss: 6.7713 - val_loss: 15.7997
Epoch 6/70
178/178 [==============================] - 5s 28ms/step - loss: 6.2542 - val_loss: 15.1940
Epoch 7/70
178/178 [==============================] - 5s 27ms/step - loss: 5.8745 - val_loss: 14.5792
Epoch 

In [84]:
users = [(["cama_mesa_banho", "papelaria", "fashion_calcados"], "sao paulo"),
        (["esporte_lazer", "moveis_decoracao", "telefonia"], "rio de janeiro"),
        (None, None)]

prediction_date = "2018-08-27"

for user in users:
    recommendation, _ = mf.recommend(user[0], user[1], prediction_date, initial_recommendation_size=100, recommendation_size=10)
    print(f"Prediction for user ({user[0]})({user[1]}):\n{recommendation}\n")


Prediction for user (['cama_mesa_banho', 'papelaria', 'fashion_calcados'])(sao paulo):
['629e019a6f298a83aeecc7877964f935', '719d571299707561c34ba04ab867b32a', '2b4609f8948be18874494203496bc318', '3e4176d545618ed02f382a3057de32b4', '6a8631b72a2f8729b91514db87e771c0', '73326828aa5efe1ba096223de496f596', '5dee2c14e1989141e15d341d4c62d72a', 'e0cf79767c5b016251fe139915c59a26', 'aa280035c50ba62c746480a59045eec4', '437c05a395e9e47f9762e677a7068ce7']

Prediction for user (['esporte_lazer', 'moveis_decoracao', 'telefonia'])(rio de janeiro):
['6a8631b72a2f8729b91514db87e771c0', 'e0cf79767c5b016251fe139915c59a26', '3e4176d545618ed02f382a3057de32b4', '629e019a6f298a83aeecc7877964f935', '24c66f106f642621e524291a895c9032', '473795a355d29305c3ea6b156833adf5', '54d9ac713e253fa1fae9c8003b011c2a', 'f9259c9e7c0f12c70f7a81409680a5ff', 'aadff88486740e0b0ebe2be6c09476ae', '921d31a1daa51460b7a95ea5f3ab64d5']

Prediction for user (None)(None):
['6a8631b72a2f8729b91514db87e771c0', 'e0cf79767c5b016251fe139915c